In [6]:
from transformers import AutoTokenizer, VisionEncoderDecoderModel

tokenizer = AutoTokenizer.from_pretrained("kha-white/manga-ocr-base")
model = VisionEncoderDecoderModel.from_pretrained("kha-white/manga-ocr-base")



In [24]:
import os
from PIL import Image
import torch
import torchvision.transforms as transforms
from torchvision.transforms import Resize, ToTensor
from tqdm import tqdm
import openpyxl

dirpath = "testocr/"

for i in os.listdir(dirpath):
    if i.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
        image_path = dirpath + i
        image = Image.open(image_path)
        resize = Resize((224, 224))
        image = resize(image)
        # Define the transformations
        transform = transforms.Compose([
            transforms.ToTensor(),  # Convert PIL Image to tensor
            # Add any additional transformations as needed
        ])

        # Apply the transformations
        image_tensor = transform(image)

        # Add an extra dimension to simulate batch size of 1
        image_tensor = image_tensor.unsqueeze(0)

        # Pass the image through the model
        generated_ids = model.generate(image_tensor)

        # Decode generated text
        generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

        print(i + generated_text)

B_1_1.jpg敢
B_1_110.jpg八
B_1_111.jpg日
B_1_125.jpgナ
B_1_126.jpg何
B_1_2.jpg!!
B_1_60.jpg談
B_4_142.jpg2


In [25]:
def jap_ocr(image_path):
    image = Image.open(image_path)
    resize = Resize((224, 224))
    image = resize(image)
    image_tensor = transform(image)
    image_tensor = image_tensor.unsqueeze(0)
    
    generated_ids = model.generate(image_tensor)
    generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return generated_text

In [32]:
# image_folder_name = 'seg_letter'
image_folder_path = '../../pilot data/data/'# dir/dir/image

workbook = openpyxl.Workbook()
sheet = workbook.active
transform = transforms.Compose([
            transforms.ToTensor(),  # Convert PIL Image to tensor
            # Add any additional transformations as needed
        ])
result = []
for img_dir_name in tqdm(os.listdir(image_folder_path)): 
    for filename in os.listdir(image_folder_path + img_dir_name): 
        if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            image_path = os.path.join(image_folder_path,img_dir_name, filename)
            text = jap_ocr(image_path)
            result.append((img_dir_name,filename,text))
    
for (img_dir_name,filename,text) in result:
    
    sheet.append((img_dir_name,filename,text))
        
workbook.save("../new_pilotdata_ocr.xlsx")
workbook.close()

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [05:55<00:00, 44.49s/it]
